# Validation and cross-validation

In this exercise you will implement a validation pipeline.

At the end of the MSLE exercise you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2024-10-15 10:23:28--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1 [following]
--2024-10-15 10:23:29--  https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd1a1d6d1ad8ae3ca8fb5ae3759.dl.dropboxusercontent.com/cd/0/inline/CcdShraZPBiRs2DMJhTyth4kIrVQhPG74-fqNrr-h-BxUHsMnqJPb-ydP04YG5oFUnS65Yrwh3HZ-A9xpIionjkNgSDrz_J6NYC21iGQ2rTkLJDeerK4SnNUdB_v9WnC2jI/file?dl=1# [following]
--2024-10-15 10:23:30--  https://ucd1a1d6d1ad8ae3ca8fb5ae3759.dl.dropboxusercontent.com/cd/0/i

In [2]:
from typing import Tuple

import numpy as np
import pandas as pd
from sklearn import preprocessing

np.random.seed(357)

In [4]:
def load(name: str) -> Tuple[np.ndarray, np.array]:
    data = pd.read_csv(name)
    x = data.loc[:, data.columns != 'cena'].to_numpy()
    y = data['cena'].to_numpy()

    return x, y

In [8]:
x_train, y_train = load('mieszkania.csv')
x_test, y_test = load('mieszkania_test.csv')

x_test, y_test

(array([[71, 'wolowo', 2, 2, 1912, 1],
        [45, 'mokotowo', 1, 1, 1938, 0],
        [38, 'mokotowo', 1, 1, 1999, 1],
        ...,
        [89, 'wolowo', 2, 2, 1922, 1],
        [40, 'wolowo', 1, 1, 1959, 0],
        [68, 'grodziskowo', 2, 1, 1927, 0]], dtype=object),
 array([ 322227,  295878,  306530,  553641,  985348,  695726,   99751,
         891261,  536499,  527093,  861472,  701472,  429776,  547725,
         669560,  318362, 1140170,  341242,  113580,  456093,  470730,
         421012,  617318,  796117,  138901,  857820,  939450,  398165,
         944399, 1025413,  522440,  344346,  145702,  246712,  574154,
         807608,  568048,  412494,  588840,  766040,  979540, 1044803,
         742235,  758936,  388672,  178238,  530053, 1150687,  587013,
         269316,  270969, 1008103,  299708,  393925,  511106,  947932,
         127717,  752428, 1185932,  330988,  330699,  403778,  584561,
         795392,  602356,  680512,  202121,  888872,  456054,  227841,
         343730,  

In [11]:
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(x_train[:, 1])
x_train[:, 1] = labelencoder.transform(x_train[:, 1])
x_test[:, 1] = labelencoder.transform(x_test[:, 1])

x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

x_train
x_test

array([[7.100e+01, 3.000e+00, 2.000e+00, 2.000e+00, 1.912e+03, 1.000e+00],
       [4.500e+01, 1.000e+00, 1.000e+00, 1.000e+00, 1.938e+03, 0.000e+00],
       [3.800e+01, 1.000e+00, 1.000e+00, 1.000e+00, 1.999e+03, 1.000e+00],
       ...,
       [8.900e+01, 3.000e+00, 2.000e+00, 2.000e+00, 1.922e+03, 1.000e+00],
       [4.000e+01, 3.000e+00, 1.000e+00, 1.000e+00, 1.959e+03, 0.000e+00],
       [6.800e+01, 0.000e+00, 2.000e+00, 1.000e+00, 1.927e+03, 0.000e+00]])

In [ ]:
#######################################################
# TODO: Implement the basic validation method,        #
# compare MSLE on training, validation, and test sets #
#######################################################


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)

Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.

In [ ]:
####################################
# TODO: Implement cross-validation #
####################################

Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [ ]:
##############################
# TODO: Investigate the data #
##############################